In [4]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


In [5]:
boston_data = grid.search("#boston", "#data", verbose=False, return_counter=False)
boston_target = grid.search("#boston", "#target", verbose=False, return_counter=False)

In [6]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1
model = th.nn.Linear(n_features, n_targets)
optimizer = th.optim.SGD(params=model.parameters(),lr=0.0000001)

In [7]:
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


In [8]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=4, shuffle=False, drop_last=False)

In [9]:
epochs = 10
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        model.send(data.location)
        
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))
            
            
    print('Total loss', loss_accum)

Train Epoch: 1 [0/127 (0%)]	Loss: 176164.875000
Train Epoch: 1 [80/127 (16%)]	Loss: 3321.710693
Train Epoch: 1 [160/127 (31%)]	Loss: 6845.711914
Train Epoch: 1 [240/127 (47%)]	Loss: 144.904556
Train Epoch: 1 [320/127 (63%)]	Loss: 84.200249
Train Epoch: 1 [400/127 (79%)]	Loss: 9969.619141
Train Epoch: 1 [480/127 (94%)]	Loss: 2671.954590
Total loss 584632.6335096359
Train Epoch: 2 [0/127 (0%)]	Loss: 2663.337402
Train Epoch: 2 [80/127 (16%)]	Loss: 2031.335938
Train Epoch: 2 [160/127 (31%)]	Loss: 6173.493164
Train Epoch: 2 [240/127 (47%)]	Loss: 173.049606
Train Epoch: 2 [320/127 (63%)]	Loss: 81.156303
Train Epoch: 2 [400/127 (79%)]	Loss: 7798.106934
Train Epoch: 2 [480/127 (94%)]	Loss: 3138.456787
Total loss 333255.1352519989
Train Epoch: 3 [0/127 (0%)]	Loss: 2551.894531
Train Epoch: 3 [80/127 (16%)]	Loss: 1743.522339
Train Epoch: 3 [160/127 (31%)]	Loss: 6085.834961
Train Epoch: 3 [240/127 (47%)]	Loss: 196.500687
Train Epoch: 3 [320/127 (63%)]	Loss: 80.509933
Train Epoch: 3 [400/127 (79%)]

In [12]:
dataset.datasets["bob"].